<h1>Installing Required packages</h1>

In [121]:
# !pip install psycopg2 --user

<h2>Importing Essential modules</h2>

In [200]:
import psycopg2
import os
import pandas as pd
import numpy as np

<h3>Connecting Database</h3>

In [123]:
host="localhost"
user="postgres"
password = "PoZameer88@$"
port="5432"

new_database_name = "painting"

In [124]:
conn = psycopg2.connect(host=host, user=user, password=password, port=port, dbname="postgres")
conn.autocommit=True

In [125]:
cursor = conn.cursor()

<h5>Finding and printing current database names</h5>

In [126]:
cursor.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")

In [127]:
databases = cursor.fetchall()

In [128]:
print("Databases:", end=" ")
for db in databases:
    print(db[0], end=", ")

Databases: postgres, flis, uni, lis, painting, 

<h3>Creating a new database named "painting" if it doesn't exists</h3>

In [129]:
from psycopg2 import sql
cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{new_database_name}'")
exists = cursor.fetchone()

if not exists:
    cursor.execute(sql.SQL(f"CREATE DATABASE {new_database_name}"))
    print(f"Database '{new_database_name}' created successfully.")
    
else:
    print(f"Database '{new_database_name}' already exists.")


cursor.close()
conn.close()

Database 'painting' already exists.


<h4>Getting Filenames of all csv files</h4>

In [224]:
# The given path is my current file location, Put yours in place, provided "Data" directory is at same level
current_file_location = "C:/Users/ikram/OneDrive/Desktop/Painting DA"
current_file_location

'C:/Users/ikram/OneDrive/Desktop/Painting DA'

In [131]:
import os

os.chdir(current_file_location)
os.chdir("./Data")

csv_files = []

for root, dirs, files in os.walk("./"):
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(root, file))

print(csv_files)

['./artist.csv', './canvas_size.csv', './image_link.csv', './museum.csv', './museum_hours.csv', './product_size.csv', './subject.csv', './work.csv']


<h4>Creating a connection to database using sqlalchemy</h4>

In [134]:
from sqlalchemy import create_engine
import urllib.parse

encoded_password = urllib.parse.quote_plus(password)
db_url = f'postgresql+psycopg2://{user}:{encoded_password}@{host}:{port}/{new_database_name}'
engine = create_engine(db_url)

connection = engine.connect()

<h4>Loading <code>artist</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [132]:
artist = pd.read_csv(csv_files[0])
artist.shape

(421, 9)

In [133]:
artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_id     421 non-null    int64 
 1   full_name     421 non-null    object
 2   first_name    421 non-null    object
 3   middle_names  148 non-null    object
 4   last_name     421 non-null    object
 5   nationality   421 non-null    object
 6   style         421 non-null    object
 7   birth         421 non-null    int64 
 8   death         421 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 29.7+ KB


In [135]:
artist.to_sql(name="artist", con=connection, if_exists="replace", index=False)

421

<h4>Loading <code>canvas_size</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [136]:
canvas_size = pd.read_csv(csv_files[1])
canvas_size.shape

(200, 4)

In [137]:
canvas_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   size_id  200 non-null    int64  
 1   width    200 non-null    int64  
 2   height   193 non-null    float64
 3   label    200 non-null    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 6.4+ KB


In [138]:
canvas_size.head(2)

,size_id,width,height,label
0,20,20,NaN,"20"" Long Edge"
1,24,24,NaN,"24"" Long Edge"


In [139]:
canvas_size.to_sql(name="canvas_size", con=connection, if_exists="replace", index=False)

200

<h4>Loading <code>image_links</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [140]:
image_links = pd.read_csv(csv_files[2])
image_links.shape

(14775, 4)

In [141]:
image_links.head(2)

,work_id,url,thumbnail_small_url,thumbnail_large_url
0,181978,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
1,173188,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...


In [142]:
image_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14775 entries, 0 to 14774
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   work_id              14775 non-null  int64 
 1   url                  14775 non-null  object
 2   thumbnail_small_url  14773 non-null  object
 3   thumbnail_large_url  14773 non-null  object
dtypes: int64(1), object(3)
memory usage: 461.8+ KB


In [143]:
image_links.to_sql(name="image_links", con=connection, if_exists="replace", index=False)

775

<h4>Loading <code>museum</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [144]:
museum = pd.read_csv(csv_files[3])
museum.shape

(57, 9)

In [145]:
museum.head(2)

,museum_id,name,address,city,state,postal,country,phone,url
0,30,The Museum of Modern Art,11 W 53rd St,New York,NY,10019,USA,+1 212 708-9400,https://www.moma.org/
1,31,Pushkin State Museum of Fine Arts,12 Ulitsa Volkhonka,Moscow,NaN,119019,Russia,+7 495 697-95-78,https://pushkinmuseum.art/


In [146]:
museum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   museum_id  57 non-null     int64 
 1   name       57 non-null     object
 2   address    57 non-null     object
 3   city       57 non-null     object
 4   state      38 non-null     object
 5   postal     50 non-null     object
 6   country    57 non-null     object
 7   phone      57 non-null     object
 8   url        57 non-null     object
dtypes: int64(1), object(8)
memory usage: 4.1+ KB


In [147]:
museum.to_sql(name="museum", con=connection, if_exists="replace", index=False)

57

<h4>Loading <code>museum_hours</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [187]:
museum_hours = pd.read_csv(csv_files[4])
museum_hours.shape

(351, 4)

In [188]:
museum_hours.head(2)

,museum_id,day,open,close
0,30,Sunday,10:30:AM,05:30:PM
1,30,Monday,10:30:AM,05:30:PM


In [189]:
museum_hours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   museum_id  351 non-null    int64 
 1   day        351 non-null    object
 2   open       351 non-null    object
 3   close      351 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.1+ KB


In [190]:
museum_hours["open"].unique()

array(['10:30:AM', '11:00:AM', '10:00:AM', '09:30:AM', '09:00:AM',
       '04:00:PM', '01:00:PM', '12:30:PM', '11:30:AM', '08:15:AM',
       '12:00:PM'], dtype=object)

In [191]:
museum_hours["close"].unique()

array(['05:30:PM', '07:00:PM', '06:00:PM', '05:00:PM', '08:00:PM',
       '09:00:PM', '06:30:PM', '09:45:PM', '02:00:PM', '04:00:PM',
       '05:15:PM', '08:45:PM', '04:30:PM', '03:30:PM', '08 :00:PM'],
      dtype=object)

In [192]:
museum_hours["close"].replace({"08 :00:PM": "08:00:PM"}, inplace=True)

C:\Users\ikram\AppData\Local\Temp\ipykernel_8436\819222061.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  museum_hours["close"].replace({"08 :00:PM": "08:00:PM"}, inplace=True)


In [193]:
museum_hours.to_sql(name="museum_hours", con=connection, if_exists="replace", index=False)

351

<h4>Loading <code>product_size</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [207]:
product_size = pd.read_csv(csv_files[5])
product_size.shape

(110347, 4)

In [208]:
product_size.head(2)

,work_id,size_id,sale_price,regular_price
0,160228,24,85,85
1,160228,30,95,95


In [209]:
product_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110347 entries, 0 to 110346
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   work_id        110347 non-null  int64 
 1   size_id        110347 non-null  object
 2   sale_price     110347 non-null  int64 
 3   regular_price  110347 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 3.4+ MB


In [213]:
product_size["size_id"] = pd.to_numeric(product_size["size_id"], errors='coerce')

In [214]:
product_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110347 entries, 0 to 110346
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   work_id        110347 non-null  int64  
 1   size_id        110135 non-null  float64
 2   sale_price     110347 non-null  int64  
 3   regular_price  110347 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.4 MB


In [215]:
product_size.to_sql(name="product_size", con=connection, if_exists="replace", index=False)

347

<h4>Loading <code>subject</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [216]:
subject = pd.read_csv(csv_files[6])
subject.shape

(6771, 2)

In [217]:
subject.head(2)

,work_id,subject
0,160228,Still-Life
1,160236,Still-Life


In [218]:
subject.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771 entries, 0 to 6770
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   work_id  6771 non-null   int64 
 1   subject  6771 non-null   object
dtypes: int64(1), object(1)
memory usage: 105.9+ KB


In [219]:
subject.to_sql(name="subject", con=connection, if_exists="replace", index=False)

771

<h4>Loading <code>work</code> csv, checking basic informations and loading in postgreSQL database</h4>

In [220]:
work = pd.read_csv(csv_files[7])
work.shape

(14776, 5)

In [221]:
work.head(2)

,work_id,name,artist_id,style,museum_id
0,160228,Still Life with Flowers and a Watch,615,Baroque,43.0
1,160236,Still Life with Fruit and a Beaker on a Cock's...,615,Baroque,43.0


In [222]:
work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14776 entries, 0 to 14775
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   work_id    14776 non-null  int64  
 1   name       14776 non-null  object 
 2   artist_id  14776 non-null  int64  
 3   style      13490 non-null  object 
 4   museum_id  4553 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 577.3+ KB


In [223]:
work.to_sql(name="work", con=connection, if_exists="replace", index=False)

776